In [46]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical

# Zdefiniuj podstawową ścieżkę do danych
base_dir = 'Garbage/Garbage classification/Garbage classification'

# Funkcja do ładowania danych z pliku
def load_data(file_path):
    images = []
    labels = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(' ')
            image_name, class_label = parts[0], int(parts[1]) - 1
            image_path = os.path.join(base_dir, image_name)
            if os.path.isfile(image_path):
                images.append(image_path)
                labels.append(class_label)
            else:
                print(f"File not found: {image_path}")
    return images, np.array(labels)

# Wczytywanie danych treningowych i walidacyjnych
train_images, train_labels = load_data(os.path.join(base_dir, 'one-indexed-files-notrash_train.txt'))
val_images, val_labels = load_data(os.path.join(base_dir, 'one-indexed-files-notrash_val.txt'))

In [47]:
def data_generator(image_paths, labels, batch_size=32):
    num_samples = len(image_paths)
    while True:  # Loop forever so the generator never terminates
        for offset in range(0, num_samples, batch_size):
            batch_samples = image_paths[offset:offset+batch_size]
            batch_labels = labels[offset:offset+batch_size]

            images = []
            for batch_sample in batch_samples:
                img = load_img(batch_sample, target_size=(150, 150))
                img = img_to_array(img)
                img /= 255.0  # Normalizacja
                images.append(img)

            X_batch = np.array(images)
            y_batch = to_categorical(batch_labels, num_classes=6)  # Zakładając, że mamy 6 klas

            yield X_batch, y_batch

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Przykładowa architektura modelu
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')  # 6 klas wyjściowych
])

# Kompilacja modelu
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [49]:
# Ustawienie rozmiaru partii
batch_size = 32

# Inicjalizacja generatorów danych
train_generator = data_generator(train_images, train_labels, batch_size=batch_size)
validation_generator = data_generator(val_images, val_labels, batch_size=batch_size)

In [50]:
# Trenowanie modelu
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_images) // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(val_images) // batch_size
)

Epoch 1/10
55/55 [==============================] - 25s 441ms/step - loss: 1.6348 - accuracy: 0.3403 - val_loss: 1.4179 - val_accuracy: 0.4250
Epoch 2/10
55/55 [==============================] - 29s 528ms/step - loss: 1.3732 - accuracy: 0.4447 - val_loss: 1.3612 - val_accuracy: 0.4469
Epoch 3/10
55/55 [==============================] - 29s 525ms/step - loss: 1.2343 - accuracy: 0.5098 - val_loss: 1.2554 - val_accuracy: 0.5188
Epoch 4/10
55/55 [==============================] - 29s 529ms/step - loss: 1.1035 - accuracy: 0.5553 - val_loss: 1.3640 - val_accuracy: 0.4781
Epoch 5/10
55/55 [==============================] - 28s 511ms/step - loss: 0.9522 - accuracy: 0.6348 - val_loss: 1.3610 - val_accuracy: 0.5437
Epoch 6/10
55/55 [==============================] - 30s 547ms/step - loss: 0.8467 - accuracy: 0.6786 - val_loss: 1.2399 - val_accuracy: 0.5688
Epoch 7/10
55/55 [==============================] - 31s 551ms/step - loss: 0.7389 - accuracy: 0.7212 - val_loss: 1.3656 - val_accuracy: 0.5063